In [ ]:
import numpy as np
import pandas as pd

from matplotlib import pyplot as plt

from sklearn.datasets import fetch_olivetti_faces
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error

# Dataset

In [ ]:
faces = fetch_olivetti_faces(shuffle=True, random_state=123)

In [ ]:
print(faces.DESCR)

In [ ]:
data = faces.data
data.shape

## Wybrane zdjęcia

In [ ]:
def arr_2_img(data, i=None):
    plt.gray()
    if i is not None:
        plt.subplot(2,2,i+1)
        plt.imshow(data[i].reshape(64,64), interpolation='nearest', vmin=0, vmax=1)
    else:
        plt.imshow(data.reshape(64,64), interpolation='nearest', vmin=0, vmax=1)

arr_2_img(data, 0)
arr_2_img(data, 1)
arr_2_img(data, 2)
arr_2_img(data, 3)

# PCA

In [ ]:
pca = PCA().fit(data)

plt.figure(figsize=(9,6))
plt.plot(range(1, len(pca.explained_variance_ratio_)+1), np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance');

Przy około 100 komponentach widać załamanie krzywej na wykresie, to znaczy, że powinniśmy dobrać taki parametr `n_components`.

In [ ]:
pca_100 = PCA(n_components=100).fit(data)
data_transformed = pca_100.transform(data)

In [ ]:
data_transformed.shape

In [ ]:
compression = data.shape[1] / data_transformed.shape[1]
print('Stopien kompresji = ' + str(round(compression, 2)))

In [ ]:
retrived = pca_100.inverse_transform(data_transformed)
retrived.shape

## Porównanie oryginalnych zdjęć z odzyskanymi poprzez odwrotną transformację

In [ ]:
def show_samples(dataset, name):
    arr_2_img(dataset, 0)
    arr_2_img(dataset, 1)
    arr_2_img(dataset, 2)
    arr_2_img(dataset, 3)
    plt.suptitle(name)
    plt.show()

show_samples(retrived, "Retrived")
show_samples(data, "Original")

In [ ]:
for i in range(4):
    print('RMSE for '+ str(i) + ' photo ' + str(mean_squared_error(data[i], retrived[i], squared=False)))

In [ ]:
print('RMSE for full dataset =  ' + str(mean_squared_error(data, retrived, squared=False)))

# Przekształcenie oryginalnych zdjęć

In [ ]:
def rotate_90_add_sym(one_d_arr):
    return one_d_arr.reshape(64,64).T.reshape(4096)

def bright(one_d_arr, factor):
    return one_d_arr * factor

def flip_ud(one_d_arr):
    return np.flipud(one_d_arr.reshape(64,64)).reshape(4096)


arr_2_img(bright(data[0], 2))
plt.show()
arr_2_img(bright(data[0], 0.5))
plt.show()
arr_2_img(rotate_90_add_sym(data[0]))
plt.show()
arr_2_img(flip_ud(data[0]))

In [ ]:
rotated_data = np.array(list(map(rotate_90_add_sym,data)))
fliped_data = np.array(list(map(flip_ud,data)))
bright_data = np.array(list(map(lambda x: bright(x, 2),data)))
dark_data = np.array(list(map(lambda x: bright(x, 0.5),data)))
datas = {'rotate':rotated_data, 'flip':fliped_data, 'bright':bright_data, 'dark':dark_data}

In [ ]:
datas_retrived = {}
for name,item in datas.items():
    datas_retrived[name] = pca_100.inverse_transform(pca_100.transform(item))

In [ ]:
for name, item in datas_retrived.items():
    show_samples(item, name)
    show_samples(datas[name], 'original '+name)
    print('RMSE for "' + name + '" dataset =  ' + 
          str(mean_squared_error(datas[name], datas_retrived[name], squared=False)))

show_samples(data, "Original")

RMSE dla danych obróconych lub odbitych symetrycznie jest największe, wizualnie zdjęcia po odwrotnej transformacji też nie przypominają tych przed PCA. Co ciekawe, jasne zdjęcia też mają duże RMSE, ale spowodowane jest to wzrostem bezwzględnych wartości poszczególnych pikseli. Z tego samego powodu RMSE dla przyciemnionych zdjęć jest mniejsze niż dla oryginalnego zbioru. Aby móc porówać RMSE możemy je podzielić przez średnią wartość pikseli, wtedy powinniśmy otrzymać porównywalne wyniki.

In [ ]:
for name, item in datas_retrived.items():
    print('RMSE adjusted for brightness for "' + name + '" dataset =  ' + 
          str(mean_squared_error(datas[name], datas_retrived[name], squared=False) / np.mean(datas[name])))
    
print('RMSE adjusted for brightness for "original" dataset =  ' + 
          str(mean_squared_error(data, retrived, squared=False) / np.mean(data)))

Teraz widać, że przeskalowane względem średniej jasności RMSE jest najmniejsze dla oryginalnych obrazów, a dla obróconych jest zdecydowananie większe.

## Do czego może służyć PCA?

Ten algorytm może służyć do wykrywania niestandardowej orientacji zdjęcia i triggerować automatyczny obrót. Takie narzędzie mogłoby znaleźć zastosowanie w aparatach fotograficznych lub aplikacjach do przeglądania zdjęć. Orientacja wszystkich portretów mogłyby być automatycznie ustawiana.

## PCA losowego szumu

Byłem ciekawy jak wygląda PCA dla losowego obrazka. Poniżej widać że algorytm zapamiętał średnie wysy twarzy i dopasował do nich szum. Widać też zarys okularów.

In [ ]:
np.random.seed(123)
rand = np.random.rand(4096).reshape(1,-1)
arr_2_img(rand)
plt.show()
rand_inv = pca_100.inverse_transform(pca_100.transform(rand))
arr_2_img(rand_inv)
plt.show()
print('RMSE adjusted for brightness for "random" photo =  ' + 
          str(mean_squared_error(rand, rand_inv, squared=False) / np.mean(rand)))